Uvoz knjižnic in branje podatkov iz datotek:

In [1]:
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
list_of_cvs = ["./podatki/kd{}.csv".format(i) for i in range(2010,2020)]
frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]
df = pd.concat(frames)

C:\Users\Uporabnik\AppData\Local\Temp\ipykernel_13096\1644075804.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]
C:\Users\Uporabnik\AppData\Local\Temp\ipykernel_13096\1644075804.py:2: DtypeWarning: Columns (14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]


In [5]:
#group by mesec, count unique id
kd_leto = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD'])
leto_counter = defaultdict(int)

for name,v in kd_leto:
    index = int((name[0].split(".")[1]))
    if index >= 2010 and index < 2020:
        leto_counter[index] +=1
    


list_mes_counter = sorted(leto_counter.items())
mesec, counter = zip(*list_mes_counter)
""" plt.figure(figsize=(15,10))
plt.plot(mesec,counter)
plt.show() """

' plt.figure(figsize=(15,10))\nplt.plot(mesec,counter)\nplt.show() '

In [37]:
pijaniMladoletniki = df.groupby(['StarostniRazred', 'Skoda'])
skodaStarostStevilo = defaultdict(lambda: defaultdict(int))
for name, v in pijaniMladoletniki:
    skodaStarostStevilo[name[1]][name[0]] = len(v['ZaporednaStevilkaKD'].unique())

skodaStarostStevilo

defaultdict(<function __main__.<lambda>()>,
            {'1.000 - 10.000 EUR': defaultdict(int,
                         {'00-07': 130,
                          '07-14': 245,
                          '14-16': 172,
                          '16-18': 327,
                          '18-24': 2718,
                          '24-34': 11476,
                          '34-44': 15831,
                          '44-54': 15437,
                          '54-64': 11420,
                          'NAD 64': 8733,
                          'NI PODATKA': 45,
                          'NI POJAVA': 7,
                          'PRAVNA OSEBA': 34901}),
             '10.000 - 100.000 EUR': defaultdict(int,
                         {'00-07': 4,
                          '07-14': 26,
                          '14-16': 17,
                          '16-18': 18,
                          '18-24': 247,
                          '24-34': 1688,
                          '34-44': 2729,
                         